In [1]:
import scanpy as sc
import anndata as ad
import scanpy.external as sce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import squidpy as sq
import scipy.sparse as sp
from scipy import sparse
from scipy.sparse import csr_matrix
import math
import torch
import torch.nn as nn
import time
import torch.nn.functional as F
from itertools import chain
from scanpy import read_10x_h5
import torch.optim as optim
import sklearn
from sklearn.neighbors import kneighbors_graph
import gudhi
import networkx as nx
from torch_geometric.nn import GCNConv
import random
import os
import json 
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')

In [12]:

section_ids=['1', '2', '3', '4']
adata_l = []
for i in range(len(section_ids)):
    adata_i = sc.read_h5ad('D:\pythonplaces\SMILE-main\SMILE\datasets\dataset1\Spatial_ST'+ section_ids[i]+'_commmon_genes.h5ad')
    adata_i.obs_names = [x+'_'+section_ids[i] for x in adata_i.obs_names]
    adata_l.append(adata_i)
adata_l

[AnnData object with n_obs × n_vars = 3611 × 20091
     obs: 'in_tissue', 'array_row', 'array_col', 'batch_name', 'barcode', 'slice_id', 'layer', 'batch_name_new', 'Istrue'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'log1p', 'spatial'
     obsm: 'dist_adj', 'spatial',
 AnnData object with n_obs × n_vars = 3635 × 20091
     obs: 'in_tissue', 'array_row', 'array_col', 'batch_name', 'barcode', 'slice_id', 'layer', 'batch_name_new', 'Istrue'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'log1p', 'spatial'
     obsm: 'dist_adj', 'spatial',
 AnnData object with n_obs × n_vars = 3566 × 20091
     obs: 'in_tissue', 'array_row', 'array_col', 'batch_name', 'barcode', 'slice_id', 'layer', 'batch_name_new', 'Istrue'
     var: 'gene_ids', 'feature_types', 'genome'
     uns: 'log1p', 'spatial'
     obsm: 'dist_adj', 'spatial',
 AnnData object with n_obs × n_vars = 3431 × 20091
     obs: 'in_tissue', 'array_row', 'array_col', 'batch_name', 'barcode', 'slice_id', 'layer', 'b

In [13]:
def label_to_int(adataA, label_list, label_name):
    adata_label = np.array(adataA.obs[label_name].copy())
    for i in range(len(label_list)):
        need_index = np.where(adataA.obs[label_name]==label_list[i])[0]
        if len(need_index):
            adata_label[need_index] = i
    adataA.obs['ref'] = adata_label
    return adataA

label_list = ['L1','L2','L3','L4','L5','L6','WM']
for i in range(len(section_ids)):
    adata_l[i] = label_to_int(adata_l[i], label_list, 'layer')
adata_l[1].obs['ref'].value_counts()

ref
2    924
6    625
4    621
5    614
0    380
3    247
1    224
Name: count, dtype: int64

In [14]:
adata0_sc = sc.read_h5ad('D:\pythonplaces\SMILE-main\SMILE\datasets\dataset1\scRNA_commmon_genes.h5ad') 
adata0_sc 

AnnData object with n_obs × n_vars = 35212 × 20091
    obs: 'x', 'group', 'condition', 'celltype'
    var: 'x'
    uns: 'log1p'

In [15]:
label0_list = adata0_sc.obs['celltype'].tolist()
adata0_label_new = adata0_sc.obs['celltype'].tolist()
for i in range(len(label0_list)):
    need_index = np.where(adata0_sc.obs['celltype'] == label0_list[i])[0]    
    if len(need_index):
        for p in range(len(need_index)):
            adata0_label_new[need_index[p]] = i  

adata0_sc.obs['ref'] = pd.Series(adata0_label_new, index = adata0_sc.obs['celltype'].index)
adata0_sc.obs['ref'] = adata0_sc.obs['ref'].astype(str)
adata0_sc.obs['ref'] = adata0_sc.obs['ref'].astype('category')
adata0_sc.obs['Ground Truth'] = adata0_sc.obs['celltype']

adata0_sc.obs['ref'].unique()

['35211', '35207', '35073', '35163', '34591', ..., '33954', '35066', '35147', '6259', '32365']
Length: 33
Categories (33, object): ['28634', '32365', '33211', '33954', ..., '35208', '35209', '35211', '6259']

In [16]:
from utils import buid_adj_for_sc
adata0_sc1=buid_adj_for_sc(adata0_sc)
adata0_sc1

AnnData object with n_obs × n_vars = 35212 × 20091
    obs: 'x', 'group', 'condition', 'celltype', 'ref', 'Ground Truth'
    var: 'x'
    uns: 'log1p'
    obsp: 'adj_f'

In [17]:
adata_l.append(adata0_sc1)
len(adata_l)

5

In [18]:
adata_l1=adata_l

In [19]:
data=ad.read_h5ad('D:\pythonplaces\SMILE-main\dataset\simulation\simulated_S3_ST_final.h5ad')
data

AnnData object with n_obs × n_vars = 490 × 320
    uns: 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'proportion', 'spatial'
    varm: 'PCs'
    obsp: 'adj_f', 'adj_t', 'connectivities', 'distances'

In [ ]:
from SMILE import SMILE
in_features = len(adata_l[0].var.index)
hidden_features = 512
out_features = 50
tag_l = ['ST','ST','ST','ST','single cell']
feature_method = 'GCNConv'
alpha = 1
beta = 1
gamma = 1000
lamb = 1
theta = 0.9
spatial_regularization_strength= 0.9
lr=1e-3
subepochs=100
epochs=200
max_patience=50
min_stop=20
random_seed=2024
gpu=0
regularization_acceleration=True
edge_subset_sz=1000000
add_topology = True
add_feature = False
add_image = False
add_sc = True

multiscale = True
anchor_type = None
anchors_all = False
use_rep_anchor = 'deconvolution'
align_method = 'MMD'
anchor_size=8000
iter_comb= None
n_clusters_l = [5]
edge_weights = [1,0.1,0.1]
adata_l = SMILE(adata_l1, tag_l, section_ids, multiscale,  n_clusters_l, in_features, feature_method, 
                hidden_features, out_features, iter_comb, anchors_all, use_rep_anchor, alpha, beta, 
                lamb, theta, gamma,edge_weights, add_topology, add_feature, add_image, add_sc, 
                spatial_regularization_strength, 
                lr=0.001, subepochs=100, epochs=200,mnn_neigh = 5)